In [73]:
import pandas as pd
import csv, requests, io, json
from pathlib import Path
from datetime import datetime
from keys import POLYGON_ID, POLYGON_KEY, IEX_SECRET, IEX_PUBLISHABLE
%matplotlib inline



In [74]:
top_100_path = Path('./data/robinhood-top-100-09262020.csv')
djia_path = Path('./data/djia.csv')

In [75]:
def get_date():
    return {
        'year' : datetime.now().year,
        'month' : datetime.now().month,
        'day' : datetime.now().day,
        'hour' : datetime.now().hour,
        'minute' : datetime.now().minute
    }
date = get_date()

In [76]:
top_100 = {}
with open(top_100_path, 'r') as top_100_file:
    top_100_raw = csv.reader(top_100_file)
    next(top_100_raw)
    for line in top_100_raw:
        top_100[line[0]] = line[1]

# Data Mining
The block below is the code block for gathering the past years data on the top_100

## Alpaca.markets
This is the codeblock for data mining from the alpaca.markets API

In [146]:
for key in top_100:
    csv_path = Path(f'./data/get_top_100/{top_100[key]}.csv')
    aapl_data_response = requests.get(
        'https://data.alpaca.markets/v1/bars/1D', 
        params={
            'symbols': top_100[key],
            'start' : f"{date['year']-1}-0{date['month']}-{date['day']}T09:30:00-04:00"
        },
        headers={
            'APCA-API-KEY-ID': POLYGON_ID,
            'APCA-API-SECRET-KEY': POLYGON_KEY
        }
    )
    aapl_data_json = aapl_data_response.json()
    tckr_df = pd.DataFrame.from_records(aapl_data_json[top_100[key]])
    tckr_df = tckr_df.rename(columns={
        't' : 'date',
        'o' : 'open',
        'h' : 'high',
        'l' : 'low',
        'c' : 'close',
        'v' : 'volume'
    })
    tckr_df['date'] = pd.to_datetime(tckr_df['date'], origin='unix', unit='s')
    tckr_df.set_index(tckr_df['date'], inplace=True)
    tckr_df = tckr_df.drop('date', 1)
    tckr_df.to_csv(csv_path)

KeyError: 'date'

# Data Prep
Setting up the dataframes that I plan to work with

## DJIA Data


In [38]:
djia_df = pd.read_csv(djia_path, index_col='Date')
djia_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,28638.970703,28872.800781,28627.769531,28868.800781,28868.800781,251820000
2020-01-03,28553.330078,28716.310547,28500.359375,28634.880859,28634.880859,239590000
2020-01-06,28465.500000,28708.019531,28418.630859,28703.380859,28703.380859,252760000
2020-01-07,28639.179688,28685.500000,28565.279297,28583.679688,28583.679688,258900000
2020-01-08,28556.140625,28866.179688,28522.509766,28745.089844,28745.089844,291750000
...,...,...,...,...,...,...
2020-09-21,27484.710938,27484.710938,26715.150391,27147.699219,27147.699219,523660000
2020-09-22,27170.339844,27333.089844,26989.929688,27288.179688,27288.179688,420720000
2020-09-23,27413.599609,27464.460938,26716.119141,26763.130859,26763.130859,459470000


## Top 100 Data

In [136]:
hundred_df = {}

In [142]:
for key in top_100:
    csv_path = Path(f"./data/get_top_100/{top_100[key]}.csv")
    hundred_df[top_100[key]] = pd.read_csv(csv_path, index_col='date')

In [144]:
hundred_df['TSLA']

,open,high,low,close,volume
date,,,,,
2019-09-27 04:00:00,220.54,220.96,217.2814,218.8500,22606388
2019-09-30 04:00:00,220.90,224.58,220.7900,223.9800,22189704
2019-10-01 04:00:00,225.07,228.22,224.2000,224.5800,32017308
2019-10-02 04:00:00,223.06,223.35,217.9300,218.9600,31067943
2019-10-03 04:00:00,218.43,220.96,215.1320,220.8100,25267290
...,...,...,...,...,...
2020-09-21 04:00:00,104.54,110.19,103.1000,110.1300,178442044
2020-09-22 04:00:00,112.68,112.86,109.1600,111.8000,167185259
2020-09-23 04:00:00,111.62,112.11,106.7700,107.2300,131794926
